# UR5 ROS Manipulation

Detailed information will be found in readme file on: 
https://app.theconstructsim.com/en/catalog/course/#/Desktop

Basically, you'll need to go through the following 4 main topics:
- MoveIt
- Motion Planning
- Perception
- Grasping

### MoveIt!

MoveIt is a set of packages and tools that allow you to perform manipulation with ROS. MoveIt provides software and tools in order to do Motion Planning, Manipulation, Perception, Kinematics, Collision Checking, Control, and Navigation. 

It is a huge and very useful tool, and it is basic knowledge if you want to learn about ROS Manipulation. 

You can learn more about it by checking all of its documentation on the official website: http://moveit.ros.org

Set the robot arm in MoveIt!

In [ ]:
roslaunch ur_e_gazebo ur5e.launch limited:=true
roslaunch ur5_e_moveit_config ur5_e_moveit_planning_execution.launch sim:=true limited:=true
roslaunch ur5_e_moveit_config moveit_rviz.launch config:=true

### Motion Planning
Motion planning bascially means to plan a movement (motion) from point A to point B, without colliding with anything.

In other words, you will need to be able to control the different joints and links of your robot, avoiding collisions between them or with other elements in the environment.

create a specific python file "basic_concepts.py" to control the UR5 arm movement

In [ ]:
#! /usr/bin/env python

import sys
import copy
import rospy
import moveit_commander
import moveit_msgs.msg
import geometry_msgs.msg

moveit_commander.roscpp_initialize(sys.argv)
rospy.init_node('move_group_python_interface_tutorial', anonymous=True)

robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()    
group = moveit_commander.MoveGroupCommander("arm")
display_trajectory_publisher = rospy.Publisher('/move_group/display_planned_path', moveit_msgs.msg.DisplayTrajectory, queue_size=1)

group_variable_values = group.get_current_joint_values()

group_variable_values[1] = -1.5
group.set_joint_value_target(group_variable_values)

plan2 = group.plan()

rospy.sleep(5)
group.go(wait=True)
rospy.sleep(5)

group_variable_values[2] = 1.5
group.set_joint_value_target(group_variable_values)

plan2 = group.plan()

rospy.sleep(5)
group.go(wait=True)
rospy.sleep(5)

moveit_commander.roscpp_shutdown()

Run the script:

In [ ]:
rosrun ur5control basic_concepts.py

<img src="./Images/6_moveit_manip1.png">

Carefull!: verify the name of planning_group=manipulator

### Perception
In order to be able to interact with any object in the environment, you first need to be able to visualize it. You need to know where it is, how it is, etc... and that's what Perception is for!

Perception is usually done using RGBD cameras, like a Kinect. In the following Demo, you'll have a look at the data that the Kinect camera placed in the simulation is capturing.


### Grasping
Finally, you'll need to know about Grasping. And what is Grasping? Well, the word Grasping refers to the action of catching an object from the environment in order to do an action with it; for instance, change its position. Inside the Grasping process, there are other variables that take place, such as the Perception of the environment.

Even though Grasping may look like a very easy and simple task, it is not. Not at all! There are lots of variables that need to be taken into account, and there are lots of things that can go wrong!

## Motion Planning using Graphical Interfaces
### Part 1. Create a MoveIt package
This unit will show you how to create a MoveIt Package for your industrial robot. By completing this unit, you will be able to create a package that allows your robot to perform motion planning.

MoveIt is a ROS framework that allows you to perform motion planning with a specific robot. And... what does this mean? Well, it basically means that it allows you to plan a movement (motion) from point A to point B, without colliding with anything.

Fortunately, MoveIt provides a very nice and easy-to-use GUI, which will help us to be able to interact with the robot in order to perform motion planning. However, before being able to actually use MoveIt, we need to build a package. This package will generate all the configuration and launch files required for using our defined robot (the one that is defined in the URDF file) with MoveIt.

In order to generate this package, just follow all the steps described in the following exercise!

Install the ROS package:
https://github.com/RobotnikAutomation/summit_xl_common/tree/melodic-devel

#### Generating a MoveIt! configuration package using the Setup Assistant tool
First of all, you'll need to launch the MoveIt Setup Assistant. You can do that by typing the following command:

In [ ]:
roslaunch moveit_setup_assistant setup_assistant.launch

#### Loading your robot's URDF file
Click on the "Create New MoveIt Configuration Package" button

Now, just click the "Browse" button, select the URDF file named ur5e_joint_limited_robot.urdf.xacro in "ur_e_description" folder

and click on the "Load Files" button.

#### Define the self-collision matrix
Go to the "Self-Collisions" tab and click on the "Regenerate Collision Matrix" button. 

Here, you are just defining some pairs of links that don't need to be considered when performing collision checks. For instance, because they are adjacent links, they will always collide.

<img src="./Images/6_moveit3.png">

#### Define virtual joints
Virtual joints are used to attach the robot to the world. Basically, this means that you will create an "imaginary" joint that will connect the base of your robot with the world, so that you don't have your robotic arm floating in space.

Define a new virtual joint named virtual_joint, with base_link (the base of the robot) as Child Link, and world (the frame that represents the world) as Parent Frame. Set the Joint Type to fixed, since the base of the robot will be fixed (it won't move) on the world. Check the below images for reference:

<img src="./Images/6_ur5_moveit5.png" width="600" />

#### Define Planning Groups
Open the "Planning Groups" tab and click the "Add Group" button. Now, you will create a new group called "arm", which will use the KDLKinematicsPlugin. Just like this:

<img src="./Images/6_arm0.png" width="500" />

Next, you will click on the "Add Joints" button, and you will select (from the Available Joints box) all the joints that form the arm of the robot, excluding the gripper. Just like this:

<img src="./Images/6_ur5_moveit3.png" width="800" />

Finally, click the "Save" button

So now, you've defined a group of joints for performing Motion Planning with, and you've defined the plugin you want to use to calculate those Plans.

#### Define Robot Poses
Now, you are going to create a couple of predefined poses for your robot arm. And what are these predefined poses for? Well, basically, MoveIt will store these poses (specifically the position of the arm joints) so that you can set them as goals at any time, with just a couple of clicks.

This is useful, for instance, when there are known poses that your robot will have to visit often. However, do not worry too much about this, since it will make much more sense once you finish your MoveIt package and start to plan motions with it.

Let's then generate the poses. You will have to define the positions of the joints that will be related to each pose, as well as the planning group they will be related to. You can have a look at the below images to see how to set them:

The first pose will be named home, and all its joint positions will be zero degrees
The 2nd Pose will be named start, and its joints positions will be the following:

<img src="./Images/6_ur5_moveit8.png" width="800" />

#### Define End Effector
The next step would be to define the End Effector of our arm. For our specific case, though, you might have already noticed that our arm doesn't have an end effector. 

For that, just go the End Effectors tab, and click on the "Add End Effector" button. We will name our End Effector hand.

<img src="./Images/6_moveit_arm.png" width="800" />

#### Setup ROS Controllers
Next, we will define the ROS Controllers that will allow us to interact (and execute the motions) within the Gazebo simulation. 

Basically, we need to know the name of the ROS Controllers that will allow us to move the arm in the simulation. For this, let's execute the "Auto Add_FollowJointsTrajectory Controllers for each Planning Group" tap:

<img src="./Images/6_moveit7.png" width="800" />

Basically, we now know that the name of the control is arm_controller, and that its type is FollowJointTrajectory.

Now, click on the Add Planning Group Joints button in order to associate the controller to a planning group. Here, just select the arm group you created before

<img src="./Images/6_moveit8.png">

#### Set Up Perception
In the previous section, you configured your MoveIt package to be able to plan and execute the motions on the simulated robot. However, if you are going to actuate on the simulated environment, it would be nice to know something about this environment, right? For instance, to be able to avoid possible obstacles that are in the way of the arm.

For this issue, we are going to configure the MoveIt package so that it can "see" the environment it's in. For this, we will use an RGBD camera that is mounted at the front side of the robot.

<img src="./Images/6_moveit9.png">

we have the points topic. This topic provides the PointCloud stream that the camera is generating. You can also check this using RViz with a PointCloud2 Display

* <b>sensor_plugin</b>: This parameter specifies the name of the plugin we are using in the robot.
* <b>point_cloud_topic</b>: The plugin will listen to this topic for PointCloud data.
* <b>max_range</b>: This is the distance limit, in meters, in which any points above the range will not be used for processing.
* <b>padding_offset</b>: This value will be taken into account for robot links and attached objects when filtering clouds containing the robot links (self-filtering).
* <b>padding_scale</b>: This value will also be taken into account while self-filtering.
* <b>point_subsample</b>: If the update process is slow, points can be subsampled. If we make this value greater than 1, the points will be skipped instead of processed.
* <b>filtered_cloud_topic</b>: This is the final filtered cloud topic. We will get the processed PointCloud through this topic. It can be used mainly for debugging.

#### Generate the MoveIt package
There's one last step you will need to complete before generating the MoveIt package. You need to fill the Author Information. Just complete the fields with your name and e-mail

Finally, go to the "Configuration Files" tab and click the "Browse" button. Navigate to the catkin_ws/src directory, create a new directory, and name it myur5e_moveit_config. 

Now, click the "Generate Package" button

<img src="./Images/6_moveit10.png">

And that's it! You've created your MoveIt package for your robot. But... now what?

Now that you've already created a MoveIt package, and you've worked a little bit with it, let's take a deeper look at some key aspects of Moveit.

## Basic Motion Planning
Well... to start, you can just launch the MoveIt Rviz environment and begin to do some tests about Motion Planning. So, follow the next exercise in order to do so!

In [ ]:
roslaunch myur5e_moveit_config demo.launch

<img src="./Images/6_moveit11.png">